In [2]:
import math

In [30]:
def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

def convert_to_centimeters(foot,inches):
    cm = (foot * 12 * 2.54 + inches * 2.54)
    return cm

def verify_2barrier(P, T, thickness,d,tvl_6mv_ps,tvl_18mv_ps,a_6mv,a_18mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param tvl_18mv_ps: TVL for 18 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    param a_18mv: scatter fraction for 18 MV
    '''
  
    H_l_6mv = H_leak_unshielded(1990, d) * B2(thickness, 34, 29)
    H_l_18mv = H_leak_unshielded(250, d) * B2(thickness, 36, 34)

    H_ps_6mv = H_ps_unshielded(520, d, 1, a_6mv, 1600) * B1(thickness, tvl_6mv_ps)
    H_ps_18mv = H_ps_unshielded(250, d, 1, a_18mv, 1600) * B1(thickness, tvl_18mv_ps)

    H_total = H_ps_6mv + H_ps_18mv + H_l_6mv + H_l_18mv
    
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    
    print("H_leak_6mv: ", H_l_6mv, "mSv/wk")
    print("H_leak_18mv: ", H_l_18mv, "mSv/wk")
    print("H_ps_6mv: ", H_ps_6mv, "mSv/wk")
    print("H_ps_18mv: ", H_ps_18mv, "mSv/wk")
    print("----------------------------------")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")
    print(f"distance from target to 0.3m beyond distal surface: {d} m")
    print(f"TVL_ps (6MV) = {tvl_6mv_ps} cm")
    print(f"TVL_ps (18MV) = {tvl_18mv_ps} cm")
    print(f"Scatter Fraction (6MV) = {a_6mv}")
    print(f"Scatter Fraction (18MV) = {a_18mv}")
    print("--------------------------------------")
    print(f"TVL_1 (6MV) = 34 cm")
    print(f"TVL_e (6MV) = 29 cm")
    print(f"TVL_1 (18MV) = 36 cm")
    print(f"TVL_e (18MV) = 34 cm")
# Important note: thickness must be in centimeters.

def verify_2barrier_vault2(P, T, thickness,d,tvl_6mv_ps,a_6mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    '''
  
    H_l_6mv = H_leak_unshielded(1435.5, d) * B2(thickness, 34, 29)

    H_ps_6mv = H_ps_unshielded(319, d, 1, a_6mv, 400) * B1(thickness, tvl_6mv_ps)

    H_total = H_ps_6mv + H_l_6mv 
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    print(f"H_ps_6mv: {H_ps_6mv} mSv/wk")
    print(f"H_l_6mv: {H_l_6mv} mSv/wk")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")
    print(f"TVL (6MV) = {tvl_6mv_ps} cm")
    print(f"TVL1 (6MV) = 34 cm")
    print(f"TVLe (6MV) = 29 cm")
    print(f"Scatter Fraction (6MV) = {a_6mv}")
    print(f"Workload PS (6MV) = 1435.5 Gy/wk")
    print(f"Workload Leak = 319 Gy/wk")
    print(f"distance = {d} m")


# Important note: thickness must be in centimeters.

# Verification of the point just outside the door

In [21]:
# P = 0.1 mSv/wk
# T = 1 
#  Test;  7.578673946 m
# NCRP lists occupancy outside a door as 1/8, but we can be conservtative and use T=1
thickness_IMW = convert_to_meters(2,9.5) * 100 # convert to cm
test = thickness_IMW / math.cos(math.radians(23.54)) # corrected thickness for 23.54 degrees

verify_2barrier(0.1,1, convert_to_centimeters(3,4.72927), convert_to_meters(25,6.14673), 23, 27, 1.39e-3,8.64e-4)

PASS
H_leak_6mv:  0.013256165492067325 mSv/wk
H_leak_18mv:  0.004290627399978247 mSv/wk
H_ps_6mv:  0.00151922946091619 mSv/wk
H_ps_18mv:  0.002105807636188822 mSv/wk
----------------------------------
Total dose:  0.021171829989150584 mSv/wk
Dose limit:  0.1 mSv/wk
Thickness used: 103.45234579999999 cm = 40.72926999999999 inches
distance from target to 0.3m beyond distal surface: 7.776126941999999 m
TVL_ps (6MV) = 23 cm
TVL_ps (18MV) = 27 cm
Scatter Fraction (6MV) = 0.00139
Scatter Fraction (18MV) = 0.000864
--------------------------------------
TVL_1 (6MV) = 34 cm
TVL_e (6MV) = 29 cm
TVL_1 (18MV) = 36 cm
TVL_e (18MV) = 34 cm


# Vault #1 Wall A, Secondary Barrier

In [17]:
# Worst case scenario of 33.37 deg barely grazing the edge of primary barrier
# 8.57258 m distance
#2'-3" = 2.25 ft = 68.58 cm
verify_2barrier(0.02,(1/40),convert_to_centimeters(2,8.33006),8.57258702,26,32,2.77e-3,2.53e-3)

PASS
H_leak_6mv:  0.059343853074284685 mSv/wk
H_leak_18mv:  0.014972491480161146 mSv/wk
H_ps_6mv:  0.054440377768489416 mSv/wk
H_ps_18mv:  0.09347558763793075 mSv/wk
----------------------------------
Total dose:  0.222232309960866 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Thickness used: 82.11835239999999 cm = 32.330059999999996 inches
distance from target to 0.3m beyond distal surface: 8.57258702 m
TVL_ps (6MV) = 26 cm
TVL_ps (18MV) = 32 cm
Scatter Fraction (6MV) = 0.00277
Scatter Fraction (18MV) = 0.00253
--------------------------------------
TVL_1 (6MV) = 34 cm
TVL_e (6MV) = 29 cm
TVL_1 (18MV) = 36 cm
TVL_e (18MV) = 34 cm


# Vault #1 Wall C, Secondary Barrier

In [18]:
# Worst case scenario of 33.76 deg barely grazing the edge of primary barrier
verify_2barrier(0.1,(1/2),convert_to_meters(3,6.09905) * 100,convert_to_meters(28,3.01894),26,32,2.77e-3,2.53e-3)

PASS
H_leak_6mv:  0.0082007846862334 mSv/wk
H_leak_18mv:  0.0027644039514396097 mSv/wk
H_ps_6mv:  0.00599341570237061 mSv/wk
H_ps_18mv:  0.015537926791345306 mSv/wk
----------------------------------
Total dose:  0.03249653113138892 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 106.93158700000001 cm = 42.099050000000005 inches
distance from target to 0.3m beyond distal surface: 8.611081076000001 m
TVL_ps (6MV) = 26 cm
TVL_ps (18MV) = 32 cm
Scatter Fraction (6MV) = 0.00277
Scatter Fraction (18MV) = 0.00253
--------------------------------------
TVL_1 (6MV) = 34 cm
TVL_e (6MV) = 29 cm
TVL_1 (18MV) = 36 cm
TVL_e (18MV) = 34 cm


# Vault #1 Wall B, Secondary Barrier

In [11]:
# Worst case scenario of 90 deg scattering
verify_2barrier(0.02,1,109.22,convert_to_meters(20,11.81056),17,19,4.26e-4,1.89e-4)

PASS
H_leak_6mv:  0.012394946655382993 mSv/wk
H_leak_18mv:  0.0042913633315566435 mSv/wk
H_ps_6mv:  8.146137583069772e-06 mSv/wk
H_ps_18mv:  8.24566032999638e-06 mSv/wk
----------------------------------
Total dose:  0.016702701784852704 mSv/wk
Dose limit:  0.02 mSv/wk
Thickness used: 109.22 cm = 43.0 inches
distance from target to 0.3m beyond distal surface: 6.395988224 m


# Vault #2 Wall B, Secondary Barrier

In [25]:
verify_2barrier_vault2(0.02,1,convert_to_centimeters(3,4),convert_to_meters(20,11.7),17,4.26e-4)

PASS
H_ps_6mv: 3.5099087798072497e-06 mSv/wk
H_l_6mv: 0.01638833563364694 mSv/wk
Total dose:  0.016391845542426748 mSv/wk
Dose limit:  0.02 mSv/wk
Thickness used: 101.6 cm = 40.0 inches
TVL (6MV) = 17 cm
TVL1 (6MV) = 34 cm
TVLe (6MV) = 29 cm
Scatter Fraction (6MV) = 0.000426
Workload PS (6MV) = 1435.5 Gy/wk
Workload Leak = 319 Gy/wk


# Vault # 2 Wall C, Secondary Barrier

In [8]:
# Worst case scenario of 23.99 deg scatter
# perpendicular thickness used instead of olblique thickness
verify_2barrier_vault2(0.1,(1/2), 88.9, convert_to_meters(28,9.11344,), 34, 6.73e-3)
print("-------")
verify_2barrier_vault2(0.1,(1/2), 94.615, convert_to_meters(28,9.11344,), 34, 6.73e-3)
print("-------")
verify_2barrier_vault2(0.1,(1/2), convert_to_meters(3,2.30809) * 100, convert_to_meters(28,9.11344,), 34, 6.73e-3)
print("Above calculation uses the oblique thickness.")

PASS
H_ps_6mv: 0.06784356898177737 mSv/wk
H_l_6mv: 0.02389508446374129 mSv/wk
Total dose:  0.09173865344551867 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 88.9 cm = 35.0 inches
-------
PASS
H_ps_6mv: 0.04607023749458077 mSv/wk
H_l_6mv: 0.015178875697775172 mSv/wk
Total dose:  0.061249113192355945 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 94.615 cm = 37.25 inches
-------
PASS
H_ps_6mv: 0.03840386231633575 mSv/wk
H_l_6mv: 0.012262118670151147 mSv/wk
Total dose:  0.0506659809864869 mSv/wk
Dose limit:  0.2 mSv/wk
Thickness used: 97.3025486 cm = 38.30809 inches
Above calculation uses the oblique thickness.


Our secondary barrier in Vault #1, 2'-9.5" = 88.9 cm, is not enough to shield this secondary beam. Thus, we will use 94.615 cm = 37 1/4" in the secondary barrier.

This means that we must add 3.75" of extra concrete into the Vault #2.

# Vault #1 Wall Z and IMW

In [19]:
verify_2barrier(0.1,1,convert_to_centimeters(2,6),convert_to_meters(30,11.81287),17,19,4.26e-4,1.89e-4)

PASS
H_leak_6mv:  0.07822839004653413 mSv/wk
H_leak_18mv:  0.01841920143261779 mSv/wk
H_ps_6mv:  0.00032719278829597984 mSv/wk
H_ps_18mv:  0.00020683270808119914 mSv/wk
----------------------------------
Total dose:  0.0971816169755291 mSv/wk
Dose limit:  0.1 mSv/wk
Thickness used: 76.2 cm = 30.0 inches
distance from target to 0.3m beyond distal surface: 9.444046898 m
TVL_ps (6MV) = 17 cm
TVL_ps (18MV) = 19 cm
Scatter Fraction (6MV) = 0.000426
Scatter Fraction (18MV) = 0.000189
--------------------------------------
TVL_1 (6MV) = 34 cm
TVL_e (6MV) = 29 cm
TVL_1 (18MV) = 36 cm
TVL_e (18MV) = 34 cm


# Vault #2 Wall Z and IMW

In [26]:
verify_2barrier_vault2(0.1,1,convert_to_centimeters(2,4),convert_to_meters(30,11.92294),17,4.26e-4)

PASS
H_ps_6mv: 9.979240854754437e-05 mSv/wk
H_l_6mv: 0.08441698385593906 mSv/wk
Total dose:  0.0845167762644866 mSv/wk
Dose limit:  0.1 mSv/wk
Thickness used: 71.12 cm = 28.0 inches
TVL (6MV) = 17 cm
TVL1 (6MV) = 34 cm
TVLe (6MV) = 29 cm
Scatter Fraction (6MV) = 0.000426
Workload PS (6MV) = 1435.5 Gy/wk
Workload Leak = 319 Gy/wk


# Vault #1 Ceiling Secondary Barrier

In [22]:
verify_2barrier(0.02,1/40,convert_to_centimeters(3,1.74687),convert_to_meters(16,2.59264),22,27,1.39e-3,8.64e-4)

PASS
H_leak_6mv:  0.0598744380752093 mSv/wk
H_leak_18mv:  0.017739037640129702 mSv/wk
H_ps_6mv:  0.005189316549954363 mSv/wk
H_ps_18mv:  0.009944699043527409 mSv/wk
----------------------------------
Total dose:  0.09274749130882078 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Thickness used: 95.8770498 cm = 37.746869999999994 inches
distance from target to 0.3m beyond distal surface: 4.942653056 m
TVL_ps (6MV) = 22 cm
TVL_ps (18MV) = 27 cm
Scatter Fraction (6MV) = 0.00139
Scatter Fraction (18MV) = 0.000864
--------------------------------------
TVL_1 (6MV) = 34 cm
TVL_e (6MV) = 29 cm
TVL_1 (18MV) = 36 cm
TVL_e (18MV) = 34 cm


# Vault 2 Ceiling Secondary Barrier

In [39]:
verify_2barrier_vault2(0.02,1/40,convert_to_centimeters(2,10.70118),convert_to_meters(14,6.38575),34,6.73e-3)

PASS
H_ps_6mv: 0.2797267866665436 mSv/wk
H_l_6mv: 0.09939918846500378 mSv/wk
Total dose:  0.3791259751315474 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Thickness used: 88.1409972 cm = 34.70118 inches
TVL (6MV) = 34 cm
TVL1 (6MV) = 34 cm
TVLe (6MV) = 29 cm
Scatter Fraction (6MV) = 0.00673
Workload PS (6MV) = 1435.5 Gy/wk
Workload Leak = 319 Gy/wk
distance = 4.4293980500000005 m


# Vault 2 Wall A, Secondary Barrier

In [29]:
# Worst case scenario of 21.28 deg scatter
# Used perpendicular thickness instead of oblique thickness

verify_2barrier_vault2(0.02, (1/40), convert_to_centimeters(2,2.82927) , convert_to_meters(25,2.12), 34, 6.73e-3)

PASS
H_ps_6mv: 0.3609732950023275 mSv/wk
H_l_6mv: 0.16200059571200454 mSv/wk
Total dose:  0.522973890714332 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk
Thickness used: 68.1463458 cm = 26.82927 inches
TVL (6MV) = 34 cm
TVL1 (6MV) = 34 cm
TVLe (6MV) = 29 cm
Scatter Fraction (6MV) = 0.00673
Workload PS (6MV) = 1435.5 Gy/wk
Workload Leak = 319 Gy/wk


# Verification of the point outside door in Vault 2

In [10]:
# P = 0.1 mSv/wk
# T = 1 
# Angle is 55.33 with the gantry pointing towards Wall C
# NCRP lists occupancy outside a door as 1/8, but we can be conservtative and use T=1
distance_test = convert_to_meters(25,6.17495) # m
thickness_IMW = convert_to_meters(2,11.82342) * 100 # convert to cm
test = thickness_IMW / math.cos(math.radians(55.33)) # corrected thickness for 23.54 degrees

verify_2barrier_vault2(0.1,1, thickness_IMW, distance_test, 23, 1.39e-3)

PASS
H_ps_6mv: 0.0008110490853974909 mSv/wk
H_l_6mv: 0.025714185328653116 mSv/wk
Total dose:  0.026525234414050608 mSv/wk
Dose limit:  0.1 mSv/wk
Thickness used: 90.9914868 cm = 35.82342 inches
